# 4. Modeling 

+ Logistic Regression: likely to converge to a point due to its simplistic architecture, despite how many feature engineering you did 

+ MLP: The depth of the network allows for hierarchical feature learning, which is particularly useful for complex non-linearities. Each layer the model learns new features, but it is a black box and we as humans cannot interpret what those features are

+ Tree-based algorithm: Gradient Boosting Machines (GBM) (e.g., XGBoost, LightGBM, CatBoost): These are also ensemble methods that build trees sequentially, with each new tree trying to correct the errors made by the previous ones. They are highly effective at capturing intricate non-linear patterns and often achieve state-of-the-art performance

+ Support Vector Machines with non-linear kernels: By using kernel functions (like Radial Basis Function (RBF), polynomial, or sigmoid), SVMs can implicitly map the data into a higher-dimensional space where it might become linearly separable. This allows them to learn complex non-linear decision boundaries in the original feature space

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold, train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score
import keras_tuner 
import gc
import os # For directory creation

import warnings
warnings.filterwarnings("ignore")

# --- Configuration ---
N_SPLITS = 10       # Number of folds for final CV
# N_EPOCHS = 100      # Max epochs for *final* training (can be adjusted after tuning)
TUNER_MAX_EPOCHS = 500 # Max epochs for *each trial* during tuning (Hyperband manages this)
TUNER_FACTOR = 3      # Reduction factor for Hyperband
BATCH_SIZE = 356
# L2_LAMBDA = 0.01    # Will be tuned
# DROPOUT_RATE = 0.3  # Will be tuned
EARLY_STOPPING_PATIENCE = 20 # Patience for final training AND tuner trials
RANDOM_SEED = 42

In [ ]:
train_metadata = pd.read_csv('train_processed_ver2.csv').set_index("participant_id")
test_metadata = pd.read_csv('test_processed_ver2.csv').set_index("participant_id")

train_fmri = pd.read_csv('train_fmri.csv').set_index("participant_id")
test_fmri = pd.read_csv('test_fmri.csv').set_index("participant_id")

train_merged = train_metadata.merge(train_fmri, left_index=True, right_index=True)
test_merged = test_metadata.merge(test_fmri, left_index=True, right_index=True)

y_train_df = pd.read_excel("data/TRAIN/TRAINING_SOLUTIONS.xlsx").set_index("participant_id")

X_train_df = train_merged.sort_index()
X_test_df = test_merged.sort_index()
y_train_df = y_train_df.sort_index()

# Get test participant IDs for later submission
test_ids = pd.Series(X_test_df.index, name='participant_id')

assert all(X_train_df.index == y_train_df.index), "Label IDs do not match X_train_df IDs"

In [ ]:
# --- Identify Numerical and OHE Columns (VERIFY THIS LIST) ---
numerical_cols = ['EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP', 'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan', 'Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ']
ohe_cols = [col for col in X_train_df.columns if col not in numerical_cols]

assert len(numerical_cols) + len(ohe_cols) == len(X_train_df.columns)

y = y_train_df.values

In [ ]:
# Split data for hyperparameter tuning
X_hp_train_df, X_hp_val_df, y_hp_train, y_hp_val = train_test_split(
    X_train_df, y,
    test_size=0.2, # 20% for tuner validation
    random_state=RANDOM_SEED,
    stratify=y[:, 0] # Stratify by ADHD outcome
)

print(f"HP Train shape: {X_hp_train_df.shape}, HP Val shape: {X_hp_val_df.shape}")

In [ ]:
# --- Scale Data for Hyperparameter Tuning ---
scaler_hp = StandardScaler()
X_hp_train_num_scaled = scaler_hp.fit_transform(X_hp_train_df[numerical_cols])
X_hp_val_num_scaled = scaler_hp.transform(X_hp_val_df[numerical_cols])

# Get OHE columns (no scaling)
X_hp_train_ohe = X_hp_train_df[ohe_cols].values
X_hp_val_ohe = X_hp_val_df[ohe_cols].values

# Combine for tuner input
X_hp_train_final = np.hstack((X_hp_train_num_scaled, X_hp_train_ohe))
X_hp_val_final = np.hstack((X_hp_val_num_scaled, X_hp_val_ohe))

# Prepare target dictionaries for tuner
y_hp_train_dict = {'adhd_output': y_hp_train[:, 0], 'sex_output': y_hp_train[:, 1]}
y_hp_val_dict = {'adhd_output': y_hp_val[:, 0], 'sex_output': y_hp_val[:, 1]}

In [ ]:
def build_model_for_tuning(hp):
    """Builds a tunable multi-output neural network."""
    input_shape = (X_hp_train_final.shape[1],) # Use shape from prepared data
    inputs = keras.Input(shape=input_shape)

    # --- Tune Hyperparameters ---
    hp_units_1 = hp.Int('units_1', min_value=32, max_value=128, step=16)
    hp_units_2 = hp.Int('units_2', min_value=16, max_value=64, step=16)
    hp_l2 = hp.Float('l2_lambda', min_value=1e-4, max_value=1e-1, sampling='log')
    hp_dropout = hp.Float('dropout_rate', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    x = layers.Dense(units=hp_units_1, activation='relu', kernel_regularizer=regularizers.l2(hp_l2))(inputs)
    x = layers.Dropout(rate=hp_dropout)(x)
    x = layers.Dense(units=hp_units_2, activation='relu', kernel_regularizer=regularizers.l2(hp_l2))(x)
    x = layers.Dropout(rate=hp_dropout)(x)

    output_adhd = layers.Dense(1, activation='sigmoid', name='adhd_output')(x)
    output_sex = layers.Dense(1, activation='sigmoid', name='sex_output')(x)

    model = keras.Model(inputs=inputs, outputs=[output_adhd, output_sex])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss={'adhd_output': 'binary_crossentropy', 'sex_output': 'binary_crossentropy'},
                  metrics={'adhd_output': ['accuracy'], # Keep metrics simple for tuner
                           'sex_output': ['accuracy']})
    return model

In [ ]:
# Instantiate and Run the Tuner 
tuner = keras_tuner.Hyperband(
    build_model_for_tuning,
    objective='val_loss',
    max_epochs=TUNER_MAX_EPOCHS,
    factor=TUNER_FACTOR,
    hyperband_iterations=1,
    directory='keras_tuner_dir', 
    project_name='wids_datathon_tuning',
    overwrite=True, 
    seed=RANDOM_SEED
)

tuner_early_stopping = EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE)

print("\n--- Starting Hyperparameter Search ---")
tuner.search(X_hp_train_final, y_hp_train_dict,
             epochs=TUNER_MAX_EPOCHS, # Max epochs *per trial*
             batch_size=BATCH_SIZE,
             validation_data=(X_hp_val_final, y_hp_val_dict),
             callbacks=[tuner_early_stopping],
             verbose=1 # Show progress for each trial
            )

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print("\n--- Hyperparameter Search Complete ---")
print(f"Best Hyperparameters Found:")
print(f" - Units Layer 1: {best_hps.get('units_1')}")
print(f" - Units Layer 2: {best_hps.get('units_2')}")
print(f" - L2 Lambda: {best_hps.get('l2_lambda'):.5f}")
print(f" - Dropout Rate: {best_hps.get('dropout_rate'):.2f}")
print(f" - Learning Rate: {best_hps.get('learning_rate')}")

# Clean 
# del X_hp_train_df, X_hp_val_df, y_hp_train, y_hp_val
# del X_hp_train_final, X_hp_val_final, y_hp_train_dict, y_hp_val_dict, scaler_hp
# gc.collect()

--- Hyperparameter Search Complete ---
Best Hyperparameters Found:
 - Units Layer 1: 48
 - Units Layer 2: 64
 - L2 Lambda: 0.00013
 - Dropout Rate: 0.50
 - Learning Rate: 0.001

In [ ]:
# --- 3. Weighted F1 Score Function (Same as before) ---
def calculate_weighted_f1(y_true, y_pred_proba, thresholds):
    # (Function code remains identical to previous version)
    threshold_adhd, threshold_sex = thresholds
    y_pred_adhd = (y_pred_proba[:, 0] >= threshold_adhd).astype(int)
    y_pred_sex = (y_pred_proba[:, 1] >= threshold_sex).astype(int)
    y_true_adhd = y_true[:, 0]
    y_true_sex = y_true[:, 1]
    f1_sex = f1_score(y_true_sex, y_pred_sex, zero_division=0)
    female_adhd_mask = (y_true_adhd == 1) & (y_true_sex == 1)
    sample_weights = np.ones(len(y_true_adhd))
    sample_weights[female_adhd_mask] = 2.0
    tp_adhd = np.sum(sample_weights * (y_true_adhd == 1) * (y_pred_adhd == 1))
    fp_adhd_unweighted = np.sum((y_true_adhd == 0) & (y_pred_adhd == 1))
    fn_adhd = np.sum(sample_weights * (y_true_adhd == 1) * (y_pred_adhd == 0))
    precision_adhd = tp_adhd / (tp_adhd + fp_adhd_unweighted) if (tp_adhd + fp_adhd_unweighted) > 0 else 0
    recall_adhd = tp_adhd / (tp_adhd + fn_adhd) if (tp_adhd + fn_adhd) > 0 else 0
    f1_adhd = (2 * precision_adhd * recall_adhd) / (precision_adhd + recall_adhd) if (precision_adhd + recall_adhd) > 0 else 0
    final_score = (f1_adhd + f1_sex) / 2.0
    return final_score

# --- 4. Final Model Definition (Using Best HPs) ---
# This function now builds the model with specific hyperparameters
def build_final_model(input_shape, hps):
    """Builds the final model using the best hyperparameters."""
    inputs = keras.Input(shape=input_shape)

    # Get hyperparameters from the best_hps object
    units_1 = hps.get('units_1')
    units_2 = hps.get('units_2')
    l2_lambda = hps.get('l2_lambda')
    dropout_rate = hps.get('dropout_rate')
    learning_rate = hps.get('learning_rate')

    x = layers.Dense(units=units_1, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(inputs)
    x = layers.Dropout(rate=dropout_rate)(x)
    x = layers.Dense(units=units_2, activation='relu', kernel_regularizer=regularizers.l2(l2_lambda))(x)
    x = layers.Dropout(rate=dropout_rate)(x)

    output_adhd = layers.Dense(1, activation='sigmoid', name='adhd_output')(x)
    output_sex = layers.Dense(1, activation='sigmoid', name='sex_output')(x)

    model = keras.Model(inputs=inputs, outputs=[output_adhd, output_sex])

    # Compile using the best learning rate
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss={'adhd_output': 'binary_crossentropy', 'sex_output': 'binary_crossentropy'},
                  # Include more metrics for monitoring final training if desired
                  metrics={'adhd_output': ['accuracy', tf.keras.metrics.Precision(name='prec_adhd'), tf.keras.metrics.Recall(name='rec_adhd')],
                           'sex_output': ['accuracy', tf.keras.metrics.Precision(name='prec_sex'), tf.keras.metrics.Recall(name='rec_sex')]})
    return model

In [ ]:
# --- 5. Cross-Validation and Training (Using Best HPs) ---
print("\n--- Starting Cross-Validation with Best Hyperparameters ---")
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
oof_preds_df = pd.DataFrame(np.zeros((len(X_train_df), 2)), index=X_train_df.index, columns=['ADHD_Outcome', 'Sex_F'])
fold_scores = []
X_test_scaled_list = [] # Store test predictions from each fold

# Use the full training data (X_train_df, y) for K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_df, y[:, 0])):
    print(f"\n--- Fold {fold + 1}/{N_SPLITS} ---")

    X_train_fold_df = X_train_df.iloc[train_idx]
    X_val_fold_df = X_train_df.iloc[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    # Feature Scaling (within the fold)
    scaler = StandardScaler()
    X_train_fold_num_scaled = scaler.fit_transform(X_train_fold_df[numerical_cols])
    X_val_fold_num_scaled = scaler.transform(X_val_fold_df[numerical_cols])
    X_train_fold_ohe = X_train_fold_df[ohe_cols].values
    X_val_fold_ohe = X_val_fold_df[ohe_cols].values
    X_train_fold_final = np.hstack((X_train_fold_num_scaled, X_train_fold_ohe))
    X_val_fold_final = np.hstack((X_val_fold_num_scaled, X_val_fold_ohe))

    # --- Build final model using the BEST hyperparameters found ---
    model = build_final_model(input_shape=(X_train_fold_final.shape[1],), hps=best_hps)

    # Use a suitable number of epochs for final training - could be fixed, or based on tuner results
    # Let's use a higher number than the tuner's max_epochs, but with early stopping
    FINAL_TRAINING_EPOCHS = 80
    final_early_stopping = EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, restore_best_weights=True, verbose=1)

    y_train_fold_dict = {'adhd_output': y_train_fold[:, 0], 'sex_output': y_train_fold[:, 1]}
    y_val_fold_dict = {'adhd_output': y_val_fold[:, 0], 'sex_output': y_val_fold[:, 1]}

    history = model.fit(X_train_fold_final, y_train_fold_dict,
                        epochs=FINAL_TRAINING_EPOCHS, # Train potentially longer now
                        batch_size=BATCH_SIZE,
                        validation_data=(X_val_fold_final, y_val_fold_dict),
                        callbacks=[final_early_stopping], # Use early stopping for final training
                        verbose=1)

    # Predict probabilities on validation set for OOF
    val_preds_proba = model.predict(X_val_fold_final)
    oof_preds_df.iloc[val_idx, 0] = val_preds_proba[0].flatten()
    oof_preds_df.iloc[val_idx, 1] = val_preds_proba[1].flatten()

    # Scale and predict on the test set
    X_test_num_scaled_fold = scaler.transform(X_test_df[numerical_cols])
    X_test_ohe_fold = X_test_df[ohe_cols].values
    X_test_final_fold = np.hstack((X_test_num_scaled_fold, X_test_ohe_fold))
    test_preds_fold = model.predict(X_test_final_fold)
    X_test_scaled_list.append(np.hstack(test_preds_fold))

    fold_score_temp = calculate_weighted_f1(y_val_fold, np.hstack(val_preds_proba), thresholds=(0.5, 0.5))
    print(f"Fold {fold + 1} preliminary validation score (0.5 threshold): {fold_score_temp:.4f}")
    fold_scores.append(fold_score_temp)

    del model, scaler, X_train_fold_df, X_val_fold_df, X_train_fold_final, X_val_fold_final
    gc.collect()
    keras.backend.clear_session()

In [ ]:
print(f"\nCV finished. Average preliminary score (0.5 threshold): {np.mean(fold_scores):.4f}")
oof_preds = oof_preds_df.values

In [ ]:
# --- 6. Threshold Optimization (Using OOF from Best HP models) ---
print("\n--- Optimizing Thresholds using OOF Predictions ---")
# (Code remains identical to previous version)
best_score = -1
best_thresholds = (0.5, 0.5)
threshold_space = np.linspace(0.1, 0.9, 41)
for t_adhd in threshold_space:
    for t_sex in threshold_space:
        current_score = calculate_weighted_f1(y, oof_preds, thresholds=(t_adhd, t_sex))
        if current_score > best_score:
            best_score = current_score
            best_thresholds = (t_adhd, t_sex)
            
print(f"Best OOF Weighted F1 Score: {best_score:.5f}")
print(f"Optimal Thresholds found: ADHD={best_thresholds[0]:.3f}, Sex={best_thresholds[1]:.3f}")

# --- 7. Final Test Prediction (Averaging Best HP model predictions) ---
print("\n--- Averaging Test Predictions from CV Folds ---")
final_test_preds_proba = np.mean(X_test_scaled_list, axis=0)

In [ ]:
# --- 8. Generate Submission File (Same as before) ---
print("\n--- Generating Submission File ---")
# (Code remains identical to previous version)
pred_adhd_final = (final_test_preds_proba[:, 0] >= best_thresholds[0]).astype(int)
pred_sex_final = (final_test_preds_proba[:, 1] >= best_thresholds[1]).astype(int)
submission_df = pd.DataFrame({'participant_id': test_ids, 'ADHD_Outcome': pred_adhd_final, 'Sex_F': pred_sex_final})
submission_path = 'submission_tuned.csv' # Changed filename
submission_df.to_csv(submission_path, index=False, float_format='%.1f')
print(f"Submission file saved to: {submission_path}")
print("Top 5 rows of submission file:")
print(submission_df.head())